# Lemon Agent

>[Lemon Agent](https://github.com/felixbrock/lemon-agent) 帮助您在几分钟内构建强大的 AI 助手，并可通过对 `Airtable`、`Hubspot`、`Discord`、`Notion`、`Slack` 和 `Github` 等工具进行准确可靠的读写操作来实现工作流程自动化。

请参阅[完整文档](https://github.com/felixbrock/lemonai-py-client)。


目前大多数可用的连接器都侧重于只读操作，这限制了 LLM 的潜力。另一方面，Agent 由于缺乏上下文或指令，有时会产生幻觉。

通过 `Lemon AI`，您可以让您的 Agent 访问明确定义的 API，以进行可靠的读写操作。此外，通过提供静态定义工作流程的方法，`Lemon AI` 函数还可以降低模型在不确定时产生幻觉的风险。

## 快速入门

以下快速入门指南将演示如何结合使用 Lemon AI 和 Agents 来自动化涉及与内部工具交互的工作流。

### 1. 安装 Lemon AI

需要 Python 3.8.1 及以上版本。

要在你的 Python 项目中使用 Lemon AI，请运行 `pip install lemonai`

这将安装相应的 Lemon AI 客户端，然后你可以将其导入到你的脚本中。

该工具使用 Python 包 langchain 和 loguru。如果 Lemon AI 安装有任何错误，请先安装这两个包，然后安装 Lemon AI 包。

### 2. 启动服务器

您的 Agent 以及 Lemon AI 提供的所有工具之间的交互都由 [Lemon AI Server](https://github.com/felixbrock/lemonai-server) 处理。要使用 Lemon AI，您需要在本地机器上运行服务器，以便 Lemon AI Python 客户端可以连接到它。

### 3. 将 Lemon AI 与 Langchain 结合使用

Lemon AI 会通过找到相关工具的正确组合来自动解决给定任务，或者使用 Lemon AI Functions 作为替代方案。以下示例演示了如何从 Hackernews 检索用户并将其写入 Airtable 的表格中：

#### (可选) 定义你的 Lemon AI 函数

与 [OpenAI functions](https://openai.com/blog/function-calling-and-other-api-updates) 类似，Lemon AI 提供了将工作流定义为可重用函数的功能。这些函数可以为需要尽可能接近近确定性行为的用例而定义。特定的工作流可以在单独的 lemonai.json 中定义：

```json
[
  {
    "name": "Hackernews Airtable 用户工作流",
    "description": "从 Hackernews 获取用户数据并将其追加到 Airtable 的表中",
    "tools": ["hackernews-get-user", "airtable-append-data"]
  }
]
```

你的模型将能够访问这些函数，并且在解决给定任务时会优先选择它们，而不是自行选择工具。你所要做的就是让代理知道它应该使用某个函数，只需在提示中包含该函数名即可。

#### 在你的 Langchain 项目中包含 Lemon AI

In [1]:
import os

from langchain_openai import OpenAI
from lemonai import execute_workflow

#### 加载 API 密钥和访问令牌

要使用需要身份验证的工具，您必须以 `{工具名称}_{身份验证字符串}` 的格式将相应的访问凭证存储在您的环境中，其中身份验证字符串是 API 密钥的 ["API_KEY", "SECRET_KEY", "SUBSCRIPTION_KEY", "ACCESS_KEY"] 或身份验证令牌的 ["ACCESS_TOKEN", "SECRET_TOKEN"] 之一。例如："OPENAI_API_KEY"、"BING_SUBSCRIPTION_KEY"、"AIRTABLE_ACCESS_TOKEN"。

In [2]:
""" Load all relevant API Keys and Access Tokens into your environment variables """
os.environ["OPENAI_API_KEY"] = "*INSERT OPENAI API KEY HERE*"
os.environ["AIRTABLE_ACCESS_TOKEN"] = "*INSERT AIRTABLE TOKEN HERE*"

In [ ]:
hackernews_username = "*INSERT HACKERNEWS USERNAME HERE*"
airtable_base_id = "*INSERT BASE ID HERE*"
airtable_table_id = "*INSERT TABLE ID HERE*"

""" Define your instruction to be given to your LLM """
prompt = f"""Read information from Hackernews for user {hackernews_username} and then write the results to
Airtable (baseId: {airtable_base_id}, tableId: {airtable_table_id}). Only write the fields "username", "karma"
and "created_at_i". Please make sure that Airtable does NOT automatically convert the field types.
"""

"""
Use the Lemon AI execute_workflow wrapper 
to run your Langchain agent in combination with Lemon AI  
"""
model = OpenAI(temperature=0)

execute_workflow(llm=model, prompt_string=prompt)

### 4. 提高 Agent 决策过程的透明度

为了提高 Agent 如何与 Lemon AI 工具交互以解决给定任务的透明度，所有决策、使用的工具和执行的操作都会写入本地 `lemonai.log` 文件。每次 LLM Agent 与 Lemon AI 工具堆栈交互时，都会创建一个相应的日志条目。

```log
2023-06-26T11:50:27.708785+0100 - b5f91c59-8487-45c2-800a-156eac0c7dae - hackernews-get-user
2023-06-26T11:50:39.624035+0100 - b5f91c59-8487-45c2-800a-156eac0c7dae - airtable-append-data
2023-06-26T11:58:32.925228+0100 - 5efe603c-9898-4143-b99a-55b50007ed9d - hackernews-get-user
2023-06-26T11:58:43.988788+0100 - 5efe603c-9898-4143-b99a-55b50007ed9d - airtable-append-data
```

通过使用 [Lemon AI Analytics](https://github.com/felixbrock/lemon-agent/blob/main/apps/analytics/README.md)，您可以轻松地更深入地了解工具的使用频率和顺序。作为结果，您可以识别 Agent 决策能力中的薄弱环节，并通过定义 Lemon AI 函数来转向更确定的行为。